In [ ]:
import boto3
import sagemaker
import json

from sagemaker.session import Session
sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_session.region_name

BUCKET='gps-serverless-workshop'
MODEL='sagemaker-tensorflow-2018-11-24-19-02-41-238'

In [ ]:
OUTPUT = 'ServerlessAIWorkshop/BatchTransform/output'
INPUT_BUCKET = 'sagemaker-sample-data-{}'.format(region)
DATADIR = 'batch-transform/mnist-1000-samples'

input_key = 'kmeans_batch_example/input/valid-data.csv'
input_location = 's3://{}/{}'.format(INPUT_BUCKET, DATADIR)
output_location = 's3://{}/{}'.format(BUCKET, OUTPUT)

In [ ]:
# Initialize the transformer object
transformer =sagemaker.transformer.Transformer(
    base_transform_job_name='Serverless-Workshop',
    model_name=MODEL,
    instance_count=4,
    instance_type='ml.c5.18xlarge',
    output_path=output_location
    )

In [ ]:
%%time
# To start a transform job:
transformer.transform(input_location, content_type='text/csv', split_type='Line')
# Then wait until transform job is completed
transformer.wait()

In [ ]:
s3 = boto3.resource('s3')
predictions = []
for i in range(10):
    file_key = '{}/data-{}.csv.out'.format(OUTPUT, i)

    output_obj = s3.Object(BUCKET, file_key)
    output = output_obj.get()["Body"].read().decode('utf-8')

    predictions.extend(json.loads(output)['outputs']['classes']['int64Val'])

Here, we can see the original labels are:

```
7, 2, 1, 0, 4, 1, 4, 9, 5, 9
```

Now let's print out the predictions to compare:

In [ ]:
print(', '.join(predictions))